In [12]:
import pandas as pd
import nltk
import re
import math
import timeit
from numpy import nan
import sqlalchemy

In [13]:
path = "/Users/nathan/Documents/Stats/Data Analysis/"
destination = "/Users/nathan/Documents/Stats/SignalDataScience/finalproject/datafiles/"
global debug 
debug = False
database = destination + "reviews.db"
disk_engine = sqlalchemy.create_engine('sqlite:///' + database)

In [14]:
def query_db(query, disk_engine):  
    start_time = timeit.default_timer()
    df = pd.read_sql_query(query, disk_engine)
    time_e = round((int(timeit.default_timer() - start_time))/60,2)
    if debug == True:
        print("\rQuery time: ", time_e, "minutes", end="")
    return df

In [15]:
# Make list of target product asins
query = """
SELECT x.asin, AVG(x.overall) as avg_rating, cnt FROM (
 SELECT asin, COUNT(asin) as cnt, overall 
 FROM review_full 
 GROUP BY asin
 ) x
 WHERE cnt>=500 
 GROUP BY asin
"""

target_df = query_db(query, disk_engine)
print(target_df.shape,"\n", target_df.head())
# I am arbitrarily picking my cutoff for "how many reviews a product must have"
# A better solution: look at when avg rating stabilizes (mostly) and choose that.
# Or at when sorting into 4+ star vs <4 star stabilizes

(6684, 3) 
          asin  avg_rating    cnt
0  0000031887         5.0   3776
1  000100039X         5.0   8576
2  0002007770         3.0  51281
3  0002051850         5.0   3408
4  000215725X         3.0    656


In [ ]:
query = """
select *, (
select count(*) from review_full as y 
where x.asin = y.asin & 
x.unixReviewTime >= y.unixReviewTime) as rownum 
from review_full as x WHERE rownum <= 40
"""

"""
select product, row_number, unixReviewTime
from review_full
ORDER BY unixReviewTime ASC LIMIT 40



"""
# hashtable for lookup
test_df = query_db(query, disk_engine)
print(test_df.shape)


In [5]:
query = """select count(*) from review_full as y 
where '0000031887' = y.asin & 
1260662401 >= y.unixReviewTime"""
query2 = "SELECT * FROM review_full WHERE asin = '0000031887'"
test_df = query_db(query, disk_engine)
print(test_df.shape, "\n", test_df.head())

(1, 1) 
    count(*)
0         0


In [7]:
q_piece1 = """
SELECT 
reviewerID,
overall,
unixReviewTime,
vader
FROM review_full 
WHERE asin = '
"""
q_piece2 = "' ORDER BY unixReviewTime LIMIT 40"
asin = '0000031887'
query =  q_piece1 + asin + q_piece2
test_df = query_db(query, disk_engine)
print(test_df.shape, "\n", test_df.head())


(0, 4) 
 Empty DataFrame
Columns: [reviewerID, overall, unixReviewTime, vader]
Index: []


In [22]:
# make review_subset table with first 20 reviews for each target asin
# This method is painfully slow, I can do better
# If I can make an SQL query which gives me all results at once
q_piece1 = "SELECT asin, reviewerID, overall, unixReviewTime, vader FROM review_full WHERE asin = '"
q_piece2 = "' GROUP BY reviewerID ORDER BY unixReviewTime ASC LIMIT 40"
asin_count = 0
first_loop = True
second_loop = True
for asin in target_df['asin'].tolist():
    query =  q_piece1 + str(asin) + q_piece2
    if first_loop == True:
        df = query_db(query, disk_engine)
        if df.shape[0] == 40:
            first_loop = False
            avg_rating = target_df.ix[target_df['asin']==asin, 'avg_rating']
            df['avg_rating'] = float(avg_rating)
            asin_count += 1
            print(asin_count,
                  "products included. Product review number=",
                  df.shape,
                  asin,
                  end="")
        else:
            print("fail product", asin_count, " asin:", asin)

    else:
        temp_df = query_db(query, disk_engine)
        if temp_df.shape[0] == 40:
            avg_rating = target_df.ix[target_df['asin']==asin, 'avg_rating']
            temp_df['avg_rating'] = float(avg_rating)
            df = pd.concat([df, temp_df]) 
            asin_count += 1
            print("\r",
                  asin_count,
                  "products included. Product review number=",
                  temp_df.shape,
                  asin, end="")
            if second_loop == True:
                second_loop = False
                print(df.head, df.tail)
        else:
            print("\r","fail product", asin_count, " asin:", asin, end="")

df.to_sql('review_subset',
        disk_engine,
        if_exists='replace',
        index= False,
        dtype={'asin':sqlalchemy.types.NVARCHAR(length=10),
         'overall':sqlalchemy.types.INTEGER(),
         'reviewerID':sqlalchemy.types.NVARCHAR(length=14),
         'unixReviewTime':sqlalchemy.types.INTEGER(),
         'vader':sqlalchemy.types.Float(precision=4, asdecimal=True),
         'avg_rating':sqlalchemy.types.Float(precision=3, asdecimal=True)})
print("Successfully saved to SQL database.")

 2 products included. Product review number= (40, 6) 000100039X<bound method NDFrame.head of           asin      reviewerID  overall  unixReviewTime   vader  avg_rating
0   0000031887  A1M7MWBQJEJWLY      5.0      1260662400  1.7112         5.0
1   0000031887  A3Q6CTO56DJ8UZ      4.0      1268697600  0.7826         5.0
2   0000031887  A338UR0Q3EG733      5.0      1278979200  1.6036         5.0
3   0000031887  A3H1W0DWLW5DLH      5.0      1286668800  1.2190         5.0
4   0000031887  A212HMSQUF21Z5      5.0      1293408000  1.5164         5.0
5   0000031887   AQ2NZ5X891VX1      5.0      1295308800  2.0971         5.0
6   0000031887  A2NXWW4QCL1EU1      5.0      1296604800  1.9701         5.0
7   0000031887  A12OFS8WQP86O5      5.0      1297123200  4.1629         5.0
8   0000031887  A1KLRMWW2FWPL4      5.0      1297468800  1.4596         5.0
9   0000031887  A3SBYCJ5VZQ63O      4.0      1298332800  0.2913         5.0
10  0000031887  A133IR93TLOVP2      2.0      1304208000 -0.0915        

In [23]:
# Export review_subset left join on reviewer_scores to csv

query = """
SELECT DISTINCT * FROM (review_subset a
LEFT JOIN (SELECT reviewerID, SUM(reviewerScore)
FROM reviewer_scores1245
GROUP BY reviewerID
) u
ON a.reviewerID = u.reviewerID)
"""
final_df = query_db(query, disk_engine)
print(final_df.head())
final_df.to_csv(destination + "reviewsummary.csv")

         asin      reviewerID  overall  unixReviewTime   vader  avg_rating  \
0  0000031887  A1M7MWBQJEJWLY        5      1260662400  1.7112         5.0   
1  0000031887  A3Q6CTO56DJ8UZ        4      1268697600  0.7826         5.0   
2  0000031887  A338UR0Q3EG733        5      1278979200  1.6036         5.0   
3  0000031887  A3H1W0DWLW5DLH        5      1286668800  1.2190         5.0   
4  0000031887  A212HMSQUF21Z5        5      1293408000  1.5164         5.0   

       reviewerID  SUM(reviewerScore)  
0  A1M7MWBQJEJWLY                1200  
1  A3Q6CTO56DJ8UZ                 400  
2  A338UR0Q3EG733                 200  
3  A3H1W0DWLW5DLH                4800  
4  A212HMSQUF21Z5                 200  
